#Extração de features


Um dos grandes problemas de se trabalhar com textos é sua ausência de estrutura. Os algoritmos de machine learning dependem de dados estruturados com tamanho fixo.

##Modelo Bag of words




A técnica de bag-of-words oportuniza que dados textos sejam mapeados para um vetor de tamanho fixo.



Exemplo:
<br>
 -  Documento 1: Rex foi passear pela manhã

 -  Documento 2: o cachorro Rex não gosta de passear de manhã
<br>
O vocabulário é:
*  Rex
*  foi
*  passear
*  pela
*  manhã
*  o
*  cachorro
*  não
*  gosta
*  de
*  passear

Nosso vocabulário contém 10 termos.

documento 1 -> [1,1,1,1,1,0,0,0,0,0]

documento 2 -> [1,0,1,0,1,1,1,1,1,1]



##Exemplo de código

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

documento1= 'Rex foi passear pela manhã'
documento2= 'o cachorro Rex não gosta de passear de manhã'



vectorizer = CountVectorizer() #stop_words='english'
#Converte as strings em números
count_data = vectorizer.fit_transform([documento1,documento2])
#print nomes das features
print(vectorizer.get_feature_names_out())

#print array de features
corpus=count_data.toarray()

print(f"{corpus}")



['cachorro' 'de' 'foi' 'gosta' 'manhã' 'não' 'passear' 'pela' 'rex']
[[0 0 1 0 1 0 1 1 1]
 [1 2 0 1 1 1 1 0 1]]


In [ ]:
#consulta
query="gosta"
bagquery=vectorizer.transform([query]).toarray()
print(bagquery)

[[0 0 0 1 0 0 0 0 0]]


In [ ]:
for doc in corpus:
  print(doc & bagquery)

[[0 0 0 0 0 0 0 0 0]]
[[0 0 0 1 0 0 0 0 0]]


#TF-IDF



TF-IDF representa "frequencia do termo - inverso da frequência do documento". Esta técnica quantifica o peso de uma palavra no documento, ou seja a sua importância no documento. Palavras muito frequentes na coleção, não são relevantes para recuperar os documentos por serem muito comuns.


Se o termo `casa` se repete 1 vez  em um documento com 10 termos e se repete 10 em um documento com 100 termos, podemos dizer que o termo é mais importante em um dos documentos? Dessa forma, a frequência do termo deve ser normalizada para evitar uma análise enviesada.


Por outro lado, se um termo está presente em todos os documentos da coleção, ex: `de`. Podemos dizer que ele é importante ou não?

Quando calculamos a IDF queremos medir a importância do termo na coleção. Termos muito comuns terão um baixo IDF, representando quão comuns os termos são.







**Fórmula**:



$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times (\text{IDF}(t, D))
$$

Detalhes:

TF(t, d) = # de vezes que t aparece in d / # termos in d

IDF(t, D) =  log (# documentos / #documentos que contém t)


## Exemplo:

Documento 1: "Rex foi passear pela manhã"

Documento 2: "o cachorro Rex não gosta de passear de manhã"




### Cálculo para o termo "Rex" no Documento 2:

1. **Term Frequency (TF)**:
   $$
   \text{TF}(\text{Rex}, \text{Doc2}) = \frac{1}{9} \approx 0.111
   $$
   *(9 termos no total no Documento 2)*

2. **Inverse Document Frequency (IDF)**:
   $$
   \text{IDF}(\text{Rex}, \text{corpus}) = \log\left(\frac{2}{2}\right) = 0
   $$
   *(Rex aparece em 2/2 documentos)*

3. **TF-IDF**:
   $$
   \text{TF-IDF} = \frac{1}{9} \times 0 = 0
   $$

### Cálculo para o termo "gosta" no Documento 2:

1. **Term Frequency (TF)**:
   $$
   \text{TF}(\text{gosta}, \text{Doc2}) = \frac{1}{9} \approx 0.111
   $$

2. **Inverse Document Frequency (IDF)**:
   $$
   \text{IDF}(\text{gosta}, \text{corpus}) = \log\left(\frac{2}{1}\right) \approx 0.301
   $$
   *(gosta aparece em 1/2 documentos)*

3. **TF-IDF**:
   $$
   \text{TF-IDF} = \frac{1}{9} \times 0.301 \approx 0.033
   $$

### 🔍 Interpretação:
- "Rex" tem TF-IDF = 0 porque aparece em **todos** documentos (IDF=0)
- "gosta" tem valor positivo (0.033) por ser único ao Documento 2

Documento 1: "Rex foi passear pela manhã"

Documento 2: "o cachorro Rex não gosta de passear de manhã"




| Palavra/doc | TF  | IDF          | TF*IDF |               |
|-------------|-----|--------------|--------|---------------|
| Rex         | 1/9 | log(2/2)=0   | 0      | 0             |
| passear     | 1/9 | log(2/2)=0   | 0      | 0             |
| manhã       | 1/9 | log(2/2)=0   | 0      | 0             |
| cachorro    | 1/9 | log(2/1)=0.3 | 0      | 0.11*0.3=0.03 |
| não         | 1/9 | log(2/       |        |               |
| gosta       |1/9  |              |        |               |
| passear     |1/9  |              |        |               |
| de          |2/9   |              |        |               |

##Exemplo código:

In [ ]:
import numpy as np
docs = np.array([
        'Rex foi passear de manhã',
        'o cachorro Rex não gosta de passear de manhã'
        ])



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

vectorizer = TfidfVectorizer()#stop_words=('english'))
X = vectorizer.fit_transform(docs)

feature_names = vectorizer.get_feature_names_out()
dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)


# Exemplo
1- O que poderia ser feito para reduzir a influência do termo `de` no doc 2?

2- Calcular o TF-IDF para a coleção  envolvendo discurso de ódio do [link](https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv)

3- Dividir  o conjunto em treino (70%) e teste (30%)

4- Aplicar o classificador Random Forest.






In [ ]:
#@title Bibliotecas

#4
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

In [ ]:


df = pd.read_csv('https://raw.githubusercontent.com/dbguilherme/machinelearning/main/dataset/hate_speech.csv',delimiter=";")

tfidf = TfidfVectorizer(stop_words="english",lowercase=True,max_features=5000)

X_tfidf = tfidf.fit_transform(df['tweet'])



In [ ]:
top_n = 20
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

tfidf_means = tfidf_df.mean().sort_values(ascending=False).head(top_n)

print("\nTop TF-IDF Features:\n", tfidf_means)



Top TF-IDF Features:
 bitch      0.056435
rt         0.043656
bitches    0.027465
http       0.024307
hoes       0.023703
128514     0.023390
pussy      0.022092
like       0.021559
hoe        0.020714
fuck       0.014656
ass        0.014595
don        0.014237
trash      0.014013
8220       0.013437
8221       0.013270
just       0.013235
got        0.012943
shit       0.012418
nigga      0.012217
lol        0.012087
dtype: float64


In [ ]:
y=df['class']

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3)
X_train[0]
dt = SVC()

dt = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)


# #reportando as métricas
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.09      0.16       474
           1       0.91      0.97      0.94      5670
           2       0.86      0.87      0.86      1291

    accuracy                           0.90      7435
   macro avg       0.82      0.64      0.65      7435
weighted avg       0.89      0.90      0.88      7435



# Using count

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Load data
df = pd.read_csv('https://raw.githubusercontent.com/dbguilherme/machinelearning/main/dataset/hate_speech.csv', delimiter=";")

# Use CountVectorizer instead of TfidfVectorizer
count_vectorizer = CountVectorizer(
    stop_words="english",
    lowercase=True,
    max_features=5000
)

# Get raw term counts (frequency)
X_count = count_vectorizer.fit_transform(df['tweet'])

# Show results
print("Vocabulary size:", len(count_vectorizer.vocabulary_))
print("Sample features:", list(count_vectorizer.vocabulary_.items())[:5])
df.head()

Vocabulary size: 5000
Sample features: [('rt', np.int64(3750)), ('woman', np.int64(4872)), ('shouldn', np.int64(3937)), ('complain', np.int64(1153)), ('cleaning', np.int64(1086))]


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,class,tweet
0,NaN,NaN,NaN,NaN,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,NaN,NaN,NaN,NaN,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,NaN,NaN,NaN,NaN,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,NaN,NaN,NaN,NaN,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,NaN,NaN,NaN,NaN,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
top_n = 10

count_df = pd.DataFrame(X_count.toarray(), columns=count_vectorizer.get_feature_names_out())

count_sums = count_df.sum().sort_values(ascending=False).head(top_n)


print("Top Count Features:\n", count_sums)
print("\nTop TF-IDF Features:\n", tfidf_means)


Top Count Features:
 bitch      8348
rt         7646
128514     3241
bitches    3119
http       2915
like       2787
hoes       2403
pussy      2159
hoe        1951
8220       1726
dtype: int64

Top TF-IDF Features:
 bitch      0.056435
rt         0.043656
bitches    0.027465
http       0.024307
hoes       0.023703
128514     0.023390
pussy      0.022092
like       0.021559
hoe        0.020714
fuck       0.014656
ass        0.014595
don        0.014237
trash      0.014013
8220       0.013437
8221       0.013270
just       0.013235
got        0.012943
shit       0.012418
nigga      0.012217
lol        0.012087
dtype: float64


In [ ]:
y=df['class']

X_train, X_test, y_train, y_test = train_test_split(X_count, y, test_size=0.3)

dt = SVC()

dt = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)



# #reportando as métricas
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.06      0.11       427
           1       0.92      0.97      0.94      5742
           2       0.82      0.89      0.86      1266

    accuracy                           0.90      7435
   macro avg       0.78      0.64      0.64      7435
weighted avg       0.89      0.90      0.88      7435



# Exercício:

1- Compare o TF-IDF com o tokenizador pela contagem de palavras no dataset abaixo. Use o classificador SVC.

2- Qual método foi melhor?

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

# Select categories (to keep it quick and interpretable)
classes = ['sci.space', 'rec.sport.baseball', 'comp.graphics']

# Load training and test data
train = fetch_20newsgroups(subset='train', categories=classes, remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test', categories=classes, remove=('headers', 'footers', 'quotes'))



In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X_count, y, test_size=0.3)



